<a href="https://colab.research.google.com/github/reinaldohaas/wrf_tutorial/blob/main/ecmwf2wrf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte seu Google Drive clicando no ícone de arquivos à esquerda.
#Este espaço será usado para salvar os dados Grib2 e utilizá-los em outro local.
#Note que os dados serão apagados do Colab.
#O diretório padrão é /content/drive/MyDrive/gis4wrf/datasets/met.
#Neste diretório, será criado um arquivo para cada dia que você usar para realizar a simulação.







#Defina nas seções a seguir a área de seu interesse.
#Certifique-se de que a área definida seja maior do que a que você planeja utilizar na simulação.
#Leve em consideração a direção dos ventos predominantes.
#Uma dica útil é aplicar a regra de enquadramento de fotografia 2/3 "landscape" invertida relação ao fluxo medio. Pense que quanto mais tempo esse fluxo estiver no domino da grade do modelo melhor. Porem esse raciocinio para vale para o primeiro aninhamentos. Os  internos, onde as atualizações ocorrem a cada passo de grade, essa regra pode ser relachada

Exemplo:
lat_start, lat_end, lat_step = 59, 46, 0.25

lon_start, lon_end, lon_step = -2, 12, 0.25


Isto será mostrado mo mapa abaixo

iday, imonth, iyear, ihour=15, 7, 2018, 12

eday,emonth,eyear,ehour=15,7,2018,18

In [ ]:
!pip install cdsapi

In [ ]:
!sudo apt-get install libeccodes-tools

In [ ]:
from google.colab import userdata
import requests
import os
import time

secret = userdata.get('secretWRF')

auth = (155599,secret)
datestr = '2024-05-01/2024-05-03'
timestr = ['%02d:00' % h for h in range(0, 24, 24)]

url_base = 'https://cds.climate.copernicus.eu/api/v2/'
params_common = {
    'date': datestr, 'time': timestr,
    'product_type': 'reanalysis', 'format': 'grib',
    # 'area': [-25, -56, -34, -46], # North, West, South, East
}
print(timestr)
print(params_common)


In [ ]:
# prompt: write  ~/.cdsapirc
# url: https://cds.climate.copernicus.eu/api/v2
# key: <UID>:<API key>

with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
  f.write('url: https://cds.climate.copernicus.eu/api/v2\n')
  f.write(f'key: {auth[0]}:{auth[1]}\n')


In [ ]:
# prompt: Use cdsapi na celula acima

from google.colab import userdata
import cdsapi
import os
import time

# # Monte seu Google Drive clicando no ícone de arquivos à esquerda.
# #Este espaço será usado para salvar os dados Grib2 e utilizá-los em outro local.
# #Note que os dados serão apagados do Colab.
# #O diretório padrão é /content/drive/MyDrive/gis4wrf/datasets/met.
# #Neste diretório, será criado um arquivo para cada dia que você usar para realizar a simulação.
#
#
#
#
#
#
# #Defina nas seções a seguir a área de seu interesse.
# #Certifique-se de que a área definida seja maior do que a que você planeja utilizar na simulação.
# #Leve em consideração a direção dos ventos predominantes.
# #Uma dica útil é aplicar a regra de enquadramento de fotografia 2/3 "landscape" invertida relação ao fluxo medio. Pense que quanto mais tempo esse fluxo estiver no domino da grade do modelo melhor. Porem esse raciocinio para vale para o primeiro aninhamentos. Os  internos, onde as atualizações ocorrem a cada passo de grade, essa regra pode ser relachada
#
# Exemplo:
# lat_start, lat_end, lat_step = 59, 46, 0.25
#
# lon_start, lon_end, lon_step = -2, 12, 0.25
#
#
# Isto será mostrado mo mapa abaixo
#
# iday, imonth, iyear, ihour=15, 7, 2018, 12
#
# eday,emonth,eyear,ehour=15,7,2018,18

# !pip install cdsapi
# !sudo apt-get install libeccodes-tools


c = cdsapi.Client()



c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            'geopotential', 'relative_humidity', 'temperature',
            'u_component_of_wind', 'v_component_of_wind',
        ],
        'pressure_level': [
            '1', '2', '3',
            '5', '7', '10',
            '20', '30', '50',
            '70', '100', '125',
            '150', '175', '200',
            '225', '250', '300',
            '350', '400', '450',
            '500', '550', '600',
            '650', '700', '750',
            '775', '800', '825',
            '850', '875', '900',
            '925', '950', '975',
            '1000',
        ],
        'date': datestr,
        'time': timestr,
    },
    'download.grib')

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
            '2m_temperature', 'land_sea_mask', 'mean_sea_level_pressure',
            'sea_ice_cover', 'sea_surface_temperature', 'skin_temperature', 'snow_depth',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4', 'surface_pressure', 'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4',
        ],
        'date': datestr,
        'time': timestr,
    },
    'download2.grib')


In [ ]:
# prompt: faça um código que a partir de data e uma latitude e longitudes centrais  pegue  5 graus de latitude N-S e L-o, para as próximas 32 horas de gribs em relação a data do ECMWF, a cada hora, tanto para single e level abaixo

import datetime

# Define the start date
start_date = datetime.datetime(2024, 5, 2, 0)

# Define the latitude and longitude of the center of the area of interest
center_latitude = -28
center_longitude = -52

# Define the extent of the area of interest (5 degrees in each direction)
latitude_extent = 5
longitude_extent = 5

# Calculate the start and end latitudes and longitudes
start_latitude = center_latitude - latitude_extent
end_latitude = center_latitude + latitude_extent
start_longitude = center_longitude - longitude_extent
end_longitude = center_longitude + longitude_extent

print(f"Start date: {start_date}")
print(f"Center latitude: {center_latitude}")
print(f"Center longitude: {center_longitude}")
print(f"Latitude extent: {latitude_extent}")  # 5 degrees in latitude
print(f"Longitude extent: {longitude_extent}")  # 5 degrees in longitude
print(f"Start latitude: {start_latitude}")
print(f"End latitude: {end_latitude}")
print(f"Start longitude: {start_longitude}")
print(f"End longitude: {end_longitude}")


In [ ]:
from google.colab import userdata
secretWRF=userdata.get('secretWRF')

In [ ]:
# escrever a variavel secretWRF dento de um arquivo .cdsapirc no colab tal que url:
# https://cds.climate.copernicus.eu/api/v2
# key: userid:key

with open('/root/.cdsapirc', 'w') as f:
  f.write(f'url: https://cds.climate.copernicus.eu/api/v2\nkey: {secretWRF}')


In [ ]:
cat /root/.cdsapirc

In [ ]:
pip install folium geopy


In [ ]:
import numpy as np
import folium

def format_date(year, month, day, hour):
    return f"{year:04d}{month:02d}{day:02d}{hour:02d}"

def calculate_interval(start, end, step):
    return int(np.abs(start - end) / step)

def create_map(latitude_inicial, latitude_final, longitude_inicial, longitude_final, zoom=5):
    center_lat = latitude_final + (latitude_inicial - latitude_final) / 2
    center_lon = longitude_inicial + (longitude_final - longitude_inicial) / 2
    return folium.Map(location=[center_lat, center_lon], zoom_start=zoom)

def add_rectangle_to_map(map_obj, lat_start, lon_start, lat_end, lon_end):
    folium.Rectangle([(lat_start, lon_start), (lat_end, lon_end)]).add_to(map_obj)

# Coordenadas para latitude
latitude_inicial, latitude_final, passo_latitude = -22, -36, 0.25

# Coordenadas para longitude
longitude_inicial, longitude_final, passo_longitude = -66, -43, 0.25

# Data inicial
hora_inicial, dia_inicial, mes_inicial, ano_inicial = 0, 2, 5, 2024

# Data final
hora_final, dia_final, mes_final, ano_final = 12, 3, 5, 2024

# Configurar a URL e a chave
print(f"Latitude inicial: {latitude_inicial}")
print(f"Latitude final: {latitude_final}")
# Changed 'latitude_step' to 'passo_latitude'
print(f"Latitude passo: {passo_latitude}")
print(f"Longitude inicial: {longitude_inicial}")
print(f"Longitude final: {longitude_final}")
# Changed 'longitude_step' to 'passo_longitude'
print(f"Longitude passo: {passo_longitude}")

# Initial date and time
data_inicial=format_date(ano_inicial,mes_inicial,dia_inicial,hora_inicial)
# Final date and time
data_final  = format_date(ano_final,mes_final,dia_final,hora_final)

print(f"Data e hora de início: {data_inicial }")
print(f"Data e hora de término: {data_final }")


# Calculate intervals
# Changed 'longitude_step' to 'passo_longitude'
lon_interval = calculate_interval(latitude_inicial, longitude_final, passo_longitude)
# Changed 'latitude_step' to 'passo_latitude'
lat_interval = calculate_interval(latitude_inicial, latitude_final, passo_latitude)

# Create a map
map_object = create_map(latitude_inicial, latitude_final, longitude_inicial, longitude_final)

# Add a rectangle to the map
add_rectangle_to_map(map_object, latitude_inicial, longitude_inicial, latitude_final, longitude_final)

# Save the map as an HTML
map_object

#Pegar os arquivos horarios da analise 0.25 graus no ERA5
=


In [ ]:
data = f"{data_inicial}_{data_final}"
print(data)

In [ ]:
import os
if not os.path.exists(os.path.join('/content/drive/MyDrive/gis4wrf/datasets/met/era5/', data)):
    !mkdir  /content/drive/MyDrive/gis4wrf/datasets/met/era5/{data}

In [ ]:
import cdsapi
from datetime import datetime, timedelta


# Criar uma lista de datas e horas
data_hora_lista = []
data_atual = datetime(ano_inicial, mes_inicial, dia_inicial, hora_inicial)
data_final = datetime(ano_final, mes_final, dia_final, hora_final)

while data_atual <= data_final:
    data_hora_lista.append(data_atual.strftime('%Y-%m-%d_%H:00'))
    data_atual += timedelta(hours=1)

# Inicializar o cliente CDS API
#URL = 'https://cds.climate.copernicus.eu/api/v2'
#KEY = secretWRF
c = cdsapi.Client()

# Configurar a área de interesse
area_interesse = [latitude_inicial, longitude_inicial, latitude_final, longitude_final]

# Configurar as variáveis e níveis desejados
variaveis = [
    'geopotential', 'relative_humidity', 'specific_humidity',
    'temperature', 'u_component_of_wind', 'v_component_of_wind',
]
niveis = [
    '1', '2', '3',
    '5', '7', '10',
    '20', '30', '50',
    '70', '100', '125',
    '150', '175', '200',
    '225', '250', '300',
    '350', '400', '450',
    '500', '550', '600',
    '650', '700', '750',
    '775', '800', '825',
    '850', '875', '900',
    '925', '950', '975',
    '1000',
]

# Loop para baixar os dados hora a hora
for data_hora in data_hora_lista:
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'variable': variaveis,
            'pressure_level': niveis,
            'year': data_hora[:4],
            'month': data_hora[5:7],
            'day': data_hora[8:10],
            'hour': data_hora[11:13],
            'area': area_interesse,
            'format': 'grib',
        },
        f'level_{data_hora}.grib'
    )


In [ ]:
import cdsapi
from datetime import datetime, timedelta

# Definir as datas e horas iniciais e finais
ano_inicial, mes_inicial, dia_inicial, hora_inicial = 2024, 5, 2, 0
ano_final, mes_final, dia_final, hora_final = 2024, 5, 2, 23

# Criar uma lista de horas
horas = []
data_atual = datetime(ano_inicial, mes_inicial, dia_inicial, hora_inicial)
data_final = datetime(ano_final, mes_final, dia_final, hora_final)

while data_atual <= data_final:
    horas.append(data_atual.strftime('%H:00'))
    data_atual += timedelta(hours=1)

# Inicializar o cliente CDS API
#URL = 'https://cds.climate.copernicus.eu/api/v2'
#KEY = {secretWRF}
c = cdsapi.Client()

# Configurar a área de interesse
latitude_inicial, longitude_inicial = -24, -56  # Norte e Oeste
latitude_final, longitude_final = -34, -46      # Sul e Leste
area_interesse = [latitude_inicial, longitude_inicial, latitude_final, longitude_final]

# Configurar as variáveis e níveis desejados
variaveis = [
    'geopotential', 'relative_humidity', 'specific_humidity',
    'temperature', 'u_component_of_wind', 'v_component_of_wind',
]
niveis = [
    '1', '2', '3',
    '5', '7', '10',
    '20', '30', '50',
    '70', '100', '125',
    '150', '175', '200',
    '225', '250', '300',
    '350', '400', '450',
    '500', '550', '600',
    '650', '700', '750',
    '775', '800', '825',
    '850', '875', '900',
    '925', '950', '975',
    '1000',
]

# Baixar os dados para todas as horas de uma só vez
c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'variable': variaveis,
        'pressure_level': niveis,
        'year': str(ano_inicial),
        'month': str(mes_inicial).zfill(2),
        'day': str(dia_inicial).zfill(2),
        'time': horas,  # Todas as horas
        'area': area_interesse,
        'format': 'grib',
    },
    'era5_20240502.grib'  # Nome do arquivo de saída
)

print("Download concluído: era5_20240502.grib")


In [ ]:
# prompt: Usar programa acima para trazer dados de superfície para
#  variable': [
#         '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
#         '2m_temperature', 'land_sea_mask', 'mean_sea_level_pressure',
#         'sea_ice_cover', 'sea_surface_temperature', 'skin_temperature', 'snow_depth',
#         'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_l

import datetime
from google.colab import userdata
import numpy as np
import folium
import os
import cdsapi
from datetime import datetime, timedelta

# ... (código anterior permanece o mesmo)

# Definir as datas e horas iniciais e finais
ano_inicial, mes_inicial, dia_inicial, hora_inicial = 2024, 5, 2, 0
ano_final, mes_final, dia_final, hora_final = 2024, 5, 2, 23

# Criar uma lista de horas
horas = []
data_atual = datetime(ano_inicial, mes_inicial, dia_inicial, hora_inicial)
data_final = datetime(ano_final, mes_final, dia_final, hora_final)

while data_atual <= data_final:
    horas.append(data_atual.strftime('%H:00'))
    data_atual += timedelta(hours=1)

# Inicializar o cliente CDS API
#URL = 'https://cds.climate.copernicus.eu/api/v2'
#KEY = '155599:158e151f-e874-437c-9e60-4e14a4681ad6'  # Substitua pela sua chave
c = cdsapi.Client()

# Configurar a área de interesse
latitude_inicial, longitude_inicial = -24, -56  # Norte e Oeste
latitude_final, longitude_final = -34, -46      # Sul e Leste
area_interesse = [latitude_inicial, longitude_inicial, latitude_final, longitude_final]

# Configurar as variáveis de superfície desejadas
variaveis_superficie = [

                '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                '2m_temperature', 'geopotential', 'land_sea_mask',
                'leaf_area_index_high_vegetation', 'mean_sea_level_pressure', 'sea_ice_cover',
                'sea_surface_temperature', 'snow_depth', 'soil_temperature_level_1',
                'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4',
                'soil_type', 'surface_latent_heat_flux', 'surface_pressure',
                'top_net_solar_radiation_clear_sky', 'total_precipitation', 'volumetric_soil_water_layer_1',
                'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4',
                'skin_temperature',
]
# Baixar os dados de superfície para todas as horas de uma só vez
c.retrieve(
    'reanalysis-era5-single-levels',  # Mudança para single-levels
    {
        'product_type': 'reanalysis',
        'variable': variaveis_superficie,  # Variáveis de superfície
        'year': str(ano_inicial),
        'month': str(mes_inicial).zfill(2),
        'day': str(dia_inicial).zfill(2),
        'time': horas,  # Todas as horas
        'area': area_interesse,
        'format': 'grib',
    },
    'era5_surface_20240502.grib'  # Nome do arquivo de saída
)

print("Download concluído: era5_surface_20240502.grib")


In [ ]:
!grib_copy era5_surface_20240502.grib era5_20240502.grib complete_20240502.grib

In [ ]:

%cp complete_20240502.grib /content/drive/MyDrive/gis4wrf/datasets/met/era5/{data}/.

In [ ]:
data_hora_lista

In [ ]:
!sudo apt-get install grads
!pip install git+https://github.com/meridionaljet/py3grads`
!wget ftp://ftp.cpc.ncep.noaa.gov/wd51we/wgrib/wgrib.c
!gcc wgrib.c -o wgrib
!sudo cp wgrib /usr/local/bin/
!wget ftp://ftp.cpc.ncep.noaa.gov/wd51we/wgrib.scripts/grib2ctl.pl
!sudo cp grib2ctl.pl  /usr/local/bin/.
!chmod 755 /usr/local/bin/grib2ctl.pl

In [ ]:
%%bash
export PATH=$PATH:/usr/local/bin
/usr/local/bin/grib2ctl.pl -verf complete_20240502.grib > level.ctl

In [ ]:

!cat level.ctl

In [ ]:

!gribmap  -e -i level.ctl -0

In [ ]:
!pip install git+https://github.com/meridionaljet/py3grads

In [ ]:
# not using `python-cartopy` package as it doesn't include all build dependencies
!apt-get install libproj-dev proj-bin proj-data
!apt-get install libgeos-dev

# shapely needs to be reinstalled to use the same geos install as cartopy (https://github.com/SciTools/cartopy/issues/871)
!pip uninstall -y shapely
!pip install --no-binary shapely shapely
!pip install cartopy

In [ ]:
from py3grads import Grads
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

# Abrir o arquivo de controle do GrADS
ga = Grads(verbose=False)
ga('open single.ctl')
ga('define mslp = MSLsfc')
mslp = ga.exp('mslp')
lons = ga.exp('lon')
lats = ga.exp('lat')

# Criar uma figura e um subplot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Adicionar os contornos de mslp
contour = ax.contourf(lons, lats, mslp/100, transform=ccrs.PlateCarree())

# Adicionar limites dos países
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)

plt.colorbar(contour, label='MSLP (hPa)')
plt.title('Mapa de MSLP  na Onde de Calor de 2018 na Europa')
plt.show()


In [ ]:
#""" Usar para instalar o WRF no Driver
#!wget https://github.com/WRF-CMake/WRF/releases/download/WRF-CMake-4.0/wrf-cmake-4.0-dmpar-basic-release-linux.tar.xz
#!wget https://github.com/WRF-CMake/WPS/releases/download/WPS-CMake-4.0/wps-cmake-4.0-dmpar-basic-release-linux.tar.xz
#%mkdir gis4wrf
#%mkdir gis4wrf/dist
#%mkdir gis4wrf/datasets
#%mkdir gis4wrf/datasets/geog
#%mkdir gis4wrf/datasets/met
#%mkdir gis4wrf/projects
#!tar -xvf wrf-cmake-4.0-dmpar-basic-release-linux.tar.xz
#%mv test gis4wrf/dist/WRF
#%mkdir gis4wrf/dist/WPS
#!tar -xvf wps-cmake-4.0-dmpar-basic-release-linux.tar.xz -C gis4wrf/dist/WPS
#***

In [ ]:
 %%writefile rules_file
 write "./[centre]_[date]_[hour]_[dataType]_[levelType].grib[editionNumber]";

In [ ]:
!grib_filter rules_file level.grib

In [ ]:
!grib_filter rules_file  single.grib

In [ ]:
import os

# Diretório onde os arquivos estão localizados
directory = '/content'  # Substitua pelo caminho correto

# Lista para armazenar os nomes dos arquivos
file_list = []

# Itera sobre os arquivos no diretório
for filename in os.listdir(directory):
    if filename.endswith("_pl.grib1"):
        new_filename = filename.replace('_pl.grib1', '')
        !grib_copy {new_filename}'_pl.grib1' {new_filename}'_sfc.grib1' {new_filename}'.grib1'
        !rm -rf {new_filename}'_pl.grib1' {new_filename}'_sfc.grib1'


print("Arquivos renomeados e lista de nomes gerada.")


# Copiar os arquivos para o gdrive

In [ ]:
%cd /content/
%cp *.grib1 /content/drive/MyDrive/gis4wrf/datasets/met/era5/{data}/.

In [ ]:
print("Isso será impresso na mesma linha.", end='')
print("Isso também será impresso na mesma linha.")


In [ ]:
%%writefile /content/drive/MyDrive/gis4wrf/projects/Amsterdam/run_wps//namelist.wps
&share
nocolons = .true.
max_dom = 2
start_date = '2018-07-15_12:00:00', '2018-07-15_12:00:00'
end_date = '2018-07-15_18:00:00', '2018-07-15_18:00:00'
interval_seconds = 3600
/

&geogrid
parent_id = 1, 1
parent_grid_ratio = 1, 5
i_parent_start = 1, 26
j_parent_start = 1, 26
e_we = 48, 46
e_sn = 48, 46
map_proj = 'lambert'
dx = 15000.0
dy = 15000.0
ref_lon = 4.010365796273883
ref_lat = 51.82601286689866
geog_data_res = '30s', '30s'
geog_data_path = '/content/drive/MyDrive/gis4wrf/datasets/geog'
truelat1 = 3.0
truelat2 = 7.0
stand_lon = 4.0
/

&metgrid
fg_name = 'FILE'
/


In [ ]:
%cd /content/drive/MyDrive/gis4wrf/projects/Amsterdam/run_wps/
!chmod 755  /content/drive/MyDrive/gis4wrf/dist/WPS/geogrid.exe
!/content/drive/MyDrive/gis4wrf/dist/WPS/geogrid.exe

In [ ]:
%%writefile /content/drive/MyDrive/gis4wrf/projects/Amsterdam/run_wrf/namelist.input
&time_control
run_days = 0
run_hours = 6
run_minutes = 0
run_seconds = 0
start_year = 2018, 2018
start_month = 7, 7
start_day = 15, 15
start_hour = 12, 12
end_year = 2018, 2018
end_month = 7, 7
end_day = 15, 15
end_hour = 18, 18
interval_seconds = 10800
input_from_file = .true., .true.
history_interval = 180, 60
frames_per_outfile = 1000, 1000
restart = .false.
restart_interval = 7200
io_form_history = 2
io_form_restart = 2
io_form_input = 2
io_form_boundary = 2
start_minute = 0, 0
start_second = 0, 0
end_minute = 0, 0
end_second = 0, 0
nocolons = .true.
/

&domains
use_adaptive_time_step = .true.
step_to_output_time = .true.
target_cfl = 1.2, 1.2
max_step_increase_pct = 5, 51
starting_time_step = 30, 5
max_time_step = -1, -1
min_time_step = 30, -1
time_step = 30
time_step_fract_num = 0
time_step_fract_den = 1
max_dom = 2
e_we = 48, 46
e_sn = 48, 46
e_vert = 33, 33
p_top_requested = 5000
num_metgrid_levels = 32
num_metgrid_soil_levels = 4
dx = 15000.0, 3000.0
dy = 15000.0, 3000.0
grid_id = 1, 2
parent_id = 1, 1
i_parent_start = 1, 26
j_parent_start = 1, 26
parent_grid_ratio = 1, 5
parent_time_step_ratio = 1, 5
feedback = 1
smooth_option = 0
/

&physics
physics_suite = 'CONUS'
mp_physics = -1, -1, -1
cu_physics = -1, -1, 0
ra_lw_physics = -1, -1, -1
ra_sw_physics = -1, -1, -1
bl_pbl_physics = -1, -1, -1
sf_sfclay_physics = -1, -1, -1
sf_surface_physics = -1, -1, -1
radt = 30, 30, 30
bldt = 0, 0, 0
cudt = 5, 5, 5
icloud = 1
num_land_cat = 21
sf_urban_physics = 0, 0, 0
/

&fdda
/

&dynamics
hybrid_opt = 2
w_damping = 0
diff_opt = 1, 1, 1
km_opt = 4, 4, 4
diff_6th_opt = 0, 0, 0
diff_6th_factor = 0.12, 0.12, 0.12
base_temp = 290.0
damp_opt = 3
zdamp = 5000.0, 5000.0, 5000.0
dampcoef = 0.2, 0.2, 0.2
khdif = 0, 0, 0
kvdif = 0, 0, 0
non_hydrostatic = .true., .true., .true.
moist_adv_opt = 1, 1, 1
scalar_adv_opt = 1, 1, 1
gwd_opt = 1
/

&bdy_control
spec_bdy_width = 5
specified = .true.
/

&grib2
/

&namelist_quilt
nio_tasks_per_group = 0
nio_groups = 1
/


In [ ]:
!chmod 755 /content/drive/MyDrive/gis4wrf/dist/WRF-4.0-mpi/main/real.exe
!chmod 755 /content/drive/MyDrive/gis4wrf/dist/WRF-4.0-mpi/main/wrf.exe
!/content/drive/MyDrive/gis4wrf/dist/WRF-4.0-mpi/main/real.exe
!mpirun -n 1 --allow-run-as-root --use-hwthread-cpus /content/drive/MyDrive/gis4wrf/dist/WRF-4.0-mpi/main/wrf.exe

In [ ]:
!mpirun -np 2 --allow-run-as-root --use-hwthread-cpus /content/drive/MyDrive/gis4wrf/dist/WRF-4.0-mpi/main/wrf.exe

In [ ]:
!/content/drive/MyDrive/gis4wrf/dist/WRF-4.0-mpi/main/real.exe


In [ ]:
%cd /content/drive/MyDrive/gis4wrf/dist/


In [ ]:
%ls

In [ ]:
!tar  xvf WRF-4.0-nonmpi.tar

In [ ]:
%rm  WRF-4.0-nonmpi.tar


In [ ]:
%cd ..

In [ ]:
%cd projects/Amsterdam/run_wps/


In [ ]:
!chmod 755 /content/drive/MyDrive/gis4wrf/dist/WPS-4.0-nompi/geogrid.exe
!/content/drive/MyDrive/gis4wrf/dist/WPS-4.0-nompi/geogrid.exe

In [ ]:
!chmod 755 /content/drive/MyDrive/gis4wrf/dist/WPS-4.0-nompi/ungrib.exe
!/content/drive/MyDrive/gis4wrf/dist/WPS-4.0-nompi/ungrib.exe

In [ ]:
!chmod 755 /content/drive/MyDrive/gis4wrf/dist/WPS-4.0-nompi/metgrid.exe
!/content/drive/MyDrive/gis4wrf/dist/WPS-4.0-nompi/metgrid.exe

In [ ]:
%cd ../run_wrf/


In [ ]:
!chmod 755 /content/drive/MyDrive/gis4wrf/dist/WRF-4.0-nompi/main/real.exe
!/content/drive/MyDrive/gis4wrf/dist/WRF-4.0-nompi/main/real.exe

In [ ]:
!chmod 755 /content/drive/MyDrive/gis4wrf/dist/WRF-4.0-nompi/main/wrf.exe
!/content/drive/MyDrive/gis4wrf/dist/WRF-4.0-nompi/main/wrf.exe